## ДЗ к Уроку 8.
### Дисперсионный анализ. Логистическая регрессия

### 1. Задание:

_Провести дисперсионный анализ для определения того, есть ли различия среднего роста среди взрослых футболистов, хоккеистов и штангистов. Даны значения роста в трех группах случайно выбранных спортсменов: Футболисты: 173, 175, 180, 178, 177, 185, 183, 182. Хоккеисты: 177, 179, 180, 188, 177, 172, 171, 184, 180. Штангисты: 172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170._

In [1]:
import numpy as np
k = 3
y1 = [173, 175, 180, 178, 177, 185, 183, 182]
y2 = [177, 179, 180, 188, 177, 172, 171, 184, 180]
y3 = [172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170]

In [2]:
n1 = len(y1)
n2 = len(y2)
n3 = len(y3)
n = n1 + n2 + n3
print(n1, n2, n3, n)

8 9 11 28


In [3]:
y1_mean = np.mean(y1)
y2_mean = np.mean(y2)
y3_mean = np.mean(y3)
print(y1_mean, y2_mean, y3_mean)

179.125 178.66666666666666 172.72727272727272


In [4]:
y_all = np.concatenate([y1, y2, y3])
y_all

array([173, 175, 180, 178, 177, 185, 183, 182, 177, 179, 180, 188, 177,
       172, 171, 184, 180, 172, 173, 169, 177, 166, 180, 178, 177, 172,
       166, 170])

In [5]:
y_mean = np.mean(y_all)
print(y_mean)

176.46428571428572


In [6]:
s2 = np.sum((y_all - y_mean)**2)
s2

830.9642857142854

In [7]:
s2_f = ((y1_mean - y_mean)**2) * n1 + ((y2_mean - y_mean)**2) * n2 + ((y3_mean - y_mean)**2) * n3
s2_f

253.9074675324678

In [8]:
s2_residual = np.sum((y1 - y1_mean)**2) + np.sum((y2 - y2_mean)**2) + np.sum((y3 - y3_mean)**2)
s2_residual

577.0568181818182

In [9]:
print(s2)
print(s2_f + s2_residual)

830.9642857142854
830.964285714286


In [10]:
sigma2_general = s2 / (n - 1)
sigma2_general

30.776455026455015

In [11]:
sigma2_f = s2_f / (k - 1)
sigma2_f

126.9537337662339

In [12]:
sigma2_residual = s2_residual / (n - k)
sigma2_residual

23.08227272727273

In [13]:
F_h = sigma2_f / sigma2_residual
F_h

5.500053450812598

In [14]:
df_m = k - 1 # = 2
print(df_m)
df_v = n - k # = 25
print(df_v)
F_c = 3.38 # для df_m = 2, df_v = 25 и a = 0.05
print(F_h > F_c)
# Так как F_h > F_c, различие среднего роста в трех группах может быть статистически значимо

2
25
True


In [15]:
eta2 = s2_f / s2
eta2
# Значение eta2 НЕ близко к 1 - значит, различие среднего роста в трех группах НЕ является статистически значимым

0.30555761769498

### 2. Задание:

_Прикрепить ссылку на курсовой проект на github. (см. выбор темы в ДЗ к Уроку 6)_

https://github.com/barinsm/GeekBrains/blob/Math_Stat_lesson08/imdb.ipynb